# One Table To Rule Them All: Radio

This notebook generates a table of radio components in the CDFS and ELAIS-S1 fields, according to various incarnations of the ATLAS survey. To run it, you will need a MongoDB server with the RGZ database loaded. All other data is fetched from the internet.

In the following cell, specify the MongoDB server details:

In [1]:
MONGO_HOST = 'localhost'
MONGO_PORT = 27017

In this cell, specify if you have access to a crowdastro output file (crowdastro.h5), and if so, where it is:

In [2]:
USING_CROWDASTRO = True
CROWDASTRO_PATH = 'crowdastro-swire.h5'
# To get this file, run `crowdastro import_data --ir swire`.

In this cell, specify if you have access to a CSV of the Fan et al. (2015) cross-identifications, and if so, where it is:

In [3]:
USING_FAN = True
FAN_PATH = 'J:/repos/crowdastro/data/fan_2015.csv'

Next, we will fetch the resources we need.

In [4]:
NORRIS_COMPONENTS_URI = 'http://www.atnf.csiro.au/people/rnorris/papers/n202/tab4.txt'
NORRIS_CROSS_IDENTIFICATIONS_URI = 'http://www.atnf.csiro.au/people/rnorris/papers/n202/tab6.txt'
MIDDELBERG_COMPONENTS_URI = 'http://iopscience.iop.org/article/10.1086/508275/fulltext/datafile4.txt'
MIDDELBERG_CROSS_IDENTIFICATIONS_URI = 'http://iopscience.iop.org/article/10.1086/508275/fulltext/datafile6.txt'

In [5]:
# Load Norris components.
import requests, io, astropy.io.ascii as asc, astropy.table, pandas
norris_components = astropy.table.Table.from_pandas(
    pandas.read_fwf(
        io.StringIO(
            requests.get(NORRIS_COMPONENTS_URI).text
        ),
        skiprows=[0, 2],
        header=0,
        widths=map(len, [
                '   # ',
                'Name                       ',
                'Radio RA     ',
                'Radio dec  ',
                'err(RA) ',
                'err(dec) ',
                'Peak Flux  ',
                'Int flux   ',
                'Bmaj   ',
                'Bmin   ',
                ' Bpa      ',
                ' rms  ',
            ])
    )
)
norris_components

#,Name,Radio RA,Radio dec,err(RA),err(dec),Peak Flux,Int flux,Bmaj,Bmin,Bpa,rms
str4,str26,str11,str11,float64,float64,float64,float64,float64,float64,float64,float64
C001,ATCDFS_J032602.78-284709.0,3:26:02.785,-28:47:09.0,0.78,0.73,0.7,1.38,8.3,2.6,60.8,79.3
C002,ATCDFS_J032604.15-275659.3,3:26:04.152,-27:56:59.3,0.55,0.9,0.71,1.97,11.5,7.3,-17.0,71.9
C003,ATCDFS_J032605.68-274734.4,3:26:05.685,-27:47:34.4,0.1,0.11,40.81,74.7,5.9,5.6,85.7,119.1
C004,ATCDFS_J032606.95-275332.2,3:26:06.955,-27:53:32.2,0.52,1.19,0.41,0.43,0.0,0.0,-1.0,76.7
C005,ATCDFS_J032611.47-273243.8,3:26:11.475,-27:32:43.8,0.1,0.1,69.65,110.9,5.3,3.3,89.6,156.3
C006,ATCDFS_J032613.70-281717.7,3:26:13.701,-28:17:17.7,0.57,0.79,0.48,0.54,0.0,0.0,0.0,77.7
C007,ATCDFS_J032615.48-284629.2,3:26:15.489,-28:46:29.2,0.34,0.36,0.45,0.71,0.0,0.0,0.0,66.0
C008,ATCDFS_J032615.55-280601.0,3:26:15.557,-28:06:01.0,0.3,0.47,0.73,1.06,0.0,0.0,-1.0,56.9
C009,ATCDFS_J032616.35-280014.6,3:26:16.353,-28:00:14.6,0.18,0.3,1.24,1.66,0.0,0.0,-1.0,60.1


In [7]:
# Load Norris cross-identifications.
# This table has inconsistent tabs, so we will have to convert them to "soft tabs".
def replace_tabs(s, tabstop=8):
    """Convert tabs to spaces."""
    out = ''
    upto = 0
    last = None
    for c in s:
        if c == '\t':
            # Fill up to next tabstop.
            diff = tabstop - upto % tabstop
            if diff == 0:
                diff = tabstop
            out += ' ' * diff
            upto += diff
            last = c
            continue
        
        last = c
        out += c
        upto += 1

    return out

test_input = ('S001	ATCDFS_J032602.78-284709.0	C001	            SWIRE3_J032603.15-284708.5	3:26:02.785	-28:47:09.06	1.4	33.8	21.1	-1.0	-1.0	-1.0						 	4					looks like a group in irac 1')
test_output = ('S001    ATCDFS_J032602.78-284709.0      C001                SWIRE3_J032603.15-284708.5  3:26:02.785     -28:47:09.06    1.4     33.8    21.1    -1.0    -1.0    -1.0                                                    4                                       looks like a group in irac 1')

assert test_output == replace_tabs(test_input)


norris_cross_identifications = astropy.table.Table.from_pandas(
    pandas.read_fwf(
        io.StringIO(
            '\n'.join(map(
                    lambda s: replace_tabs(s, 8),
                    requests.get(NORRIS_CROSS_IDENTIFICATIONS_URI).text.split('\r\n'))
             )
        ),
        skiprows=[0, 2],
        header=0,
        widths=[8, 32, 20, 28, 16, 16, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 16, 8, 16]
    )
)
norris_cross_identifications[700:710]

#,Name,Component,SWIRE,Radio RA,Radio dec,20cm,3.6µm,4.5µm,5.8µm,8.0µm,24µm,U,G,R,I,Z,z(sp),ID,Class,Basis
str4,str26,str19,str26,str11,str12,float64,float64,str8,str8,str8,float64,float64,float64,float64,float64,float64,float64,int64,str4,str3
S701,ATCDFS_J033529.42-285156.5,C753,SWIRE3_J033529.46-285154.5,3:35:29.424,-28:51:56.54,1.7,716.0,521.4,194.3,146.7,-1.0,20.6,18.8,17.5,16.7,16.4,--,3,--,--
S702,ATCDFS_J033530.22-281108.4,C754,SWIRE3_J033530.19-281108.5,3:35:30.229,-28:11:08.45,0.8,32.2,38.5,-1.0,56.2,376.3,99.0,99.0,99.0,99.0,99.0,--,1,AGN,b
S703,ATCDFS_J033531.02-272702.2,C755,--,3:35:31.025,-27:27:02.20,26.1,--,--,--,--,--,--,--,--,--,--,--,9,--,--
S704,ATCDFS_J033532.11-274547.6,C756,SWIRE3_J033532.11-274545.2,3:35:32.115,-27:45:47.61,2.2,164.4,84.6,55.3,40.6,-1.0,--,--,--,--,--,--,3,--,--
S705,ATCDFS_J033532.19-284801.0,C757,SWIRE3_J033532.19-284801.1,3:35:32.192,-28:48:01.08,0.9,222.5,417.8,1084.8,2999.4,10830.1,99.0,24.2,22.0,20.7,20.1,--,1,--,--
S706,ATCDFS_J033533.22-280621.8,C758,--,3:35:33.224,-28:06:21.85,0.3,--,--,--,--,--,--,--,--,--,--,--,9,--,--
S707,ATCDFS_J033533.96-273313.4,"C759,C760,C761,C764",SWIRE3_J033533.90-273310.9,3:35:33.963,-27:33:13.46,9.4,451.6,512.5,724.7,966.6,2208.2,--,19.8,19.5,18.9,--,--,3,AGN,b
S708,ATCDFS_J033534.47-284213.3,C763,SWIRE3_J033534.66-284221.7,3:35:34.479,-28:42:13.31,0.5,79.8,53.1,-1.0,44.3,-1.0,99.0,22.7,21.3,20.4,19.9,--,4,--,--
S709,ATCDFS_J033535.20-281729.0,"C762,C765",SWIRE3_J033535.20-281729.0,3:35:35.20,-28:17:29.00,8.4,34.5,25.3,-1.0,-1.0,-1.0,99.0,99.0,99.0,22.6,21.9,--,-1,AGN,a


In [8]:
# Load Middelberg tables.
middelberg_components = asc.read(MIDDELBERG_COMPONENTS_URI)
print(middelberg_components[0])
middelberg_cross_identifications = asc.read(MIDDELBERG_CROSS_IDENTIFICATIONS_URI)
print(middelberg_cross_identifications[0])

 ID             Name            RAh RAm  RAs  DE- DEd  DEm    DEs   e_RAs  e_DEs  PFlux IFlux MajAxis MinAxis PosAng rms  Com
                                 h  min   s       deg arcmin arcsec arcsec arcsec  mJy   mJy   arcsec  arcsec  deg   uJy     
---- -------------------------- --- --- ----- --- --- ------ ------ ------ ------ ----- ----- ------- ------- ------ ---- ---
C001 ATCDFS_J032602.78-284709.0   3  26 2.785   -  28     47    9.0   0.78   0.73   0.7  1.38     8.3     2.6   60.8 79.3  --
 ID             Name            CID            SName            RAh RAm  RAs  DE- DEd  DEm    DEs   F20 F3.6 F4.5 F5.8 F8.0 F24  umag gmag rmag Imag zmag  z  Type Class f_Class             Comm            
                                                                 h  min   s       deg arcmin arcsec mJy uJy  uJy  uJy  uJy  uJy  mag  mag  mag  mag  mag                                                     
---- -------------------------- ---- -------------------------- --- --- ----- --- --

In [9]:
# Convert Middelberg data into columns. There's no catalogue matching to do here so we can
# throw everything in right away.
import astropy.coordinates

_middelberg_component_ids = middelberg_components['ID']
_middelberg_component_names = middelberg_components['Name']
_middelberg_component_positions = [
    astropy.coordinates.SkyCoord(ra=(r['RAh'], r['RAm'], r['RAs']),
                                 dec=(-r['DEd'], r['DEm'], r['DEs']),
                                 unit=('hourangle', 'deg'))
    for r in middelberg_components
]
_middelberg_component_ras = [r.ra.deg for r in _middelberg_component_positions]
_middelberg_component_decs = [r.dec.deg for r in _middelberg_component_positions]

_middelberg_source_ids = middelberg_components['ID']
_middelberg_cid_to_source_id = {}
_middelberg_cid_to_source_name = {}
_middelberg_cid_to_swire = {}
_middelberg_cid_to_source_z = {}
_middelberg_cid_to_source_ra = {}
_middelberg_cid_to_source_dec = {}
for row in middelberg_cross_identifications:
    for component in row['CID'].split(','):
        component = component.strip()
        _middelberg_cid_to_source_id[component] = row['ID']
        _middelberg_cid_to_source_name[component] = row['Name']
        _middelberg_cid_to_swire[component] = row['SName']
        _middelberg_cid_to_source_z[component] = row['z']
        pos = astropy.coordinates.SkyCoord(ra=(row['RAh'], row['RAm'], row['RAs']),
                                           dec=(-row['DEd'], row['DEm'], row['DEs']),
                                           unit=('hourangle', 'deg'))
        _middelberg_cid_to_source_ra[component] = pos.ra.deg
        _middelberg_cid_to_source_dec[component] = pos.dec.deg

_middelberg_component_source_ids = [_middelberg_cid_to_source_id[c] for c in _middelberg_component_ids]
_middelberg_component_source_names = [_middelberg_cid_to_source_name[c] for c in _middelberg_component_ids]
_middelberg_component_swires = [_middelberg_cid_to_swire[c] for c in _middelberg_component_ids]
_middelberg_component_source_zs = [_middelberg_cid_to_source_z[c] for c in _middelberg_component_ids]
_middelberg_component_source_ras = [_middelberg_cid_to_source_ra[c] for c in _middelberg_component_ids]
_middelberg_component_source_decs = [_middelberg_cid_to_source_dec[c] for c in _middelberg_component_ids]

In [10]:
# Load RGZ.
import pymongo, numpy

client = pymongo.MongoClient(MONGO_HOST, MONGO_PORT)
db = client['radio']

_rgz_sources = []
_rgz_coords = []
_rgz_zids = []

for subject in db.radio_subjects.find({'metadata.survey': 'atlas'}):
    source = subject['metadata']['source']
    ra, dec = subject['coords']
    zid = subject['zooniverse_id']
    _rgz_sources.append(source)
    _rgz_coords.append((ra, dec))
    _rgz_zids.append(zid)

_rgz_coords = numpy.array(_rgz_coords)

In [11]:
# Load consensuses from crowdastro.
import h5py
with h5py.File(CROWDASTRO_PATH, 'r') as crowdastro_h5:
    # (atlas_i, ir_i, success, percentage)
    _crowdastro_consensus_objects = crowdastro_h5['/atlas/cdfs/consensus_objects']
    _crowdastro_zids = [r[0].decode('ascii') for r in crowdastro_h5['/atlas/cdfs/string']]
    _crowdastro_swire_names = [r.decode('ascii') for r in crowdastro_h5['/swire/cdfs/string']]
    _crowdastro_zid_to_swire = {}
    for atlas_i, ir_i, success, percentage in _crowdastro_consensus_objects:
        _crowdastro_zid_to_swire[_crowdastro_zids[int(atlas_i)]] = _crowdastro_swire_names[int(ir_i)]

In [12]:
# Match RGZ to Norris.
import scipy.spatial
_rgz_zid_to_norris = {}  # Maps ZID -> Norris CID
_norris_cids = [r['#'] for r in norris_components]
_norris_coords = [astropy.coordinates.SkyCoord(
    ra=r['Radio RA'],
    dec=r['Radio dec'],
    unit=('hourangle', 'deg')) for r in norris_components]

_norris_coords = numpy.array([(p.ra.deg, p.dec.deg) for p in _norris_coords])
_norris_tree = scipy.spatial.KDTree(_norris_coords)
# Assume that there are no situations where one Norris component maps to multiple RGZ components (and vice versa).
_dists, _indices = _norris_tree.query(_rgz_coords)
_matches = _dists < 3 / 60 / 60

for zid, match, index in zip(_rgz_zids, _matches, _indices):
    if not match:
        continue
    
    _rgz_zid_to_norris[zid] = _norris_cids[index]
_norris_to_rgz_zid = {j:i for i, j in _rgz_zid_to_norris.items()}

In [13]:
# Load Fan.
fan_cross_identifications = asc.read(FAN_PATH, header_start=0, delimiter=',')
_fan_source_ids = fan_cross_identifications['id']
_fan_id_to_swire = {r['id']:r['swire'] for r in fan_cross_identifications}
# Assuming that CID in Fan = CID in Norris.
_fan_component_to_source = {}
_fan_component_to_swire = {}
for row in fan_cross_identifications:
    components = row['radios'].split(',')
    for component in components:
        component = component.strip()
        _fan_component_to_source[component] = row['id']
        _fan_component_to_swire[component] = row['swire']

Now, we can construct the table. We will have the following columns:

- Key
- Component ID (Norris)
- Source ID (Norris)
- Source Name (Norris)
- SWIRE Name (Norris)
- RA (Norris)
- Dec (Norris)
- Source RA (Norris)
- Source Dec (Norris)
- Component ID (RGZ)
- Zooniverse ID (RGZ)
- SWIRE Name (RGZ-MV)
- RA (RGZ)
- Dec (RGZ)
- Source ID (Fan)
- SWIRE Name (Fan)
- Component ID (Middelberg)
- Component Name (Middelberg)
- RA (Middelberg)
- Dec (Middelberg)
- Source ID (Middelberg)
- Source Name (Middelberg)
- SWIRE Name (Middelberg)
- Source RA (Middelberg)
- Source Dec (Middelberg)
- Source Redshift (Middelberg)

In [14]:
columns = [
    'Key', 'Component ID (Norris)', 'Source ID (Norris)', 'Source Name (Norris)',
    'SWIRE Name (Norris)', 'RA (Norris)', 'Dec (Norris)', 'Source RA (Norris)', 'Source Dec (Norris)',
    'Component ID (RGZ)', 'Zooniverse ID (RGZ)', 'SWIRE Name (RGZ)', 'RA (RGZ)', 'Dec (RGZ)',
    'Source ID (Fan)', 'SWIRE Name (Fan)',
    'Component ID (Middelberg)', 'Component Name (Middelberg)', 'RA (Middelberg)',
    'Dec (Middelberg)', 'Source ID (Middelberg)', 'Source Name (Middelberg)',
    'SWIRE Name (Middelberg)', 'Source RA (Middelberg)', 'Source Dec (Middelberg)',
    'Source Redshift (Middelberg)',
]



In [15]:
import astropy.coordinates

# Component ID (Norris)
component_ids_norris = [r['#'] for r in norris_components]

# Source ID (Norris)
_component_to_source = {}
for r in norris_cross_identifications:
    for component in r['Component'].split(','):
        _component_to_source[component.strip()] = r['#']
source_ids_norris = [_component_to_source[c] for c in component_ids_norris]

# Source Name (Norris)
_source_to_name = {r['#']:r['Name'] for r in norris_cross_identifications}
source_names_norris = [_source_to_name[s] for s in source_ids_norris]

# SWIRE Name (Norris)
_source_to_swire_norris = {r['#']:r['SWIRE'] for r in norris_cross_identifications}
swire_names_norris = [_source_to_swire_norris[s] for s in source_ids_norris]

# RA (Norris), Dec (Norris)
_positions_norris = [astropy.coordinates.SkyCoord(
    ra=r['Radio RA'],
    dec=r['Radio dec'],
    unit=('hourangle', 'deg')) for r in norris_components]
ras_norris = [p.ra.deg for p in _positions_norris]
decs_norris = [p.dec.deg for p in _positions_norris]

# Source RA (Norris), Source Dec (Norris)
_source_positions_norris = [astropy.coordinates.SkyCoord(
    ra=r['Radio RA'],
    dec=r['Radio dec'],
    unit=('hourangle', 'deg')) for r in norris_cross_identifications]
_source_id_to_position_norris = dict(zip(norris_cross_identifications['#'], _source_positions_norris))
source_ras_norris = [_source_id_to_position_norris[s].ra.deg for s in source_ids_norris]
source_decs_norris = [_source_id_to_position_norris[s].dec.deg for s in source_ids_norris]

# Zooniverse ID (RGZ)
zooniverse_ids_rgz = [_norris_to_rgz_zid.get(cid, '') for cid in component_ids_norris]

# Component ID (RGZ)
_zid_to_cid = {z:c for z, c in zip(_rgz_zids, _rgz_sources)}
_zid_to_coord = {z:p for z, p in zip(_rgz_zids, _rgz_coords)}
component_ids_rgz = [_zid_to_cid.get(z, '') for z in zooniverse_ids_rgz]

# Extend all of these columns by RGZ objects with no corresponding Norris object.
_zid_no_norris = [z for z in _rgz_zids if z not in _rgz_zid_to_norris]
_cid_no_norris = [_zid_to_cid.get(z, '') for z in _zid_no_norris]
_blank_no_norris = [''] * len(_zid_no_norris)

for l in [component_ids_norris, source_ids_norris, source_names_norris,
          swire_names_norris, ras_norris, decs_norris, source_ras_norris,
          source_decs_norris]:
    l.extend(_blank_no_norris)
zooniverse_ids_rgz.extend(_zid_no_norris)
component_ids_rgz.extend(_cid_no_norris)

# RA (RGZ), Dec (RGZ)
ras_rgz = [_zid_to_coord.get(z, ('', ''))[0] for z in zooniverse_ids_rgz]
decs_rgz = [_zid_to_coord.get(z, ('', ''))[1] for z in zooniverse_ids_rgz]

# SWIRE Name (RGZ)
swire_names_rgz = [_crowdastro_zid_to_swire.get(z, '') for z in zooniverse_ids_rgz]

# Source ID (Fan)
fan_source_ids = [_fan_component_to_source.get(cid, '') for cid in component_ids_norris]

# SWIRE Name (Fan)
fan_swire_names = [_fan_component_to_swire.get(cid, '') for cid in component_ids_norris]

# Pad out the Middelberg columns.
middelberg_component_ids = [''] * len(component_ids_norris) + list(_middelberg_component_ids)
middelberg_component_names = [''] * len(component_ids_norris) + list(_middelberg_component_names)
middelberg_component_ras = [''] * len(component_ids_norris) + list(_middelberg_component_ras)
middelberg_component_decs = [''] * len(component_ids_norris) + list(_middelberg_component_decs)
middelberg_component_source_ids = [''] * len(component_ids_norris) + list(_middelberg_component_source_ids)
middelberg_component_source_names = [''] * len(component_ids_norris) + list(_middelberg_component_source_names)
middelberg_component_swires = [''] * len(component_ids_norris) + list(_middelberg_component_swires)
middelberg_component_source_ras = [''] * len(component_ids_norris) + list(_middelberg_component_source_ras)
middelberg_component_source_decs = [''] * len(component_ids_norris) + list(_middelberg_component_source_decs)
middelberg_component_source_zs = [''] * len(component_ids_norris) + list(_middelberg_component_source_zs)

# Pad out the other columns.
for l in [component_ids_norris, source_ids_norris, source_names_norris,
          swire_names_norris, ras_norris, decs_norris, component_ids_rgz,
          zooniverse_ids_rgz, swire_names_rgz, ras_rgz, decs_rgz,
          fan_source_ids, fan_swire_names, source_ras_norris, source_decs_norris]:
    l.extend([''] * len(_middelberg_component_ids))

keys = list(range(len(component_ids_norris)))

table = astropy.table.Table(data=[keys, component_ids_norris, source_ids_norris, source_names_norris,
                                  swire_names_norris, ras_norris, decs_norris, source_ras_norris,
                                  source_decs_norris,
                                  component_ids_rgz, zooniverse_ids_rgz, swire_names_rgz, ras_rgz, decs_rgz,
                                  fan_source_ids, fan_swire_names,
                                  middelberg_component_ids, middelberg_component_names,
                                  middelberg_component_ras, middelberg_component_decs,
                                  middelberg_component_source_ids, middelberg_component_source_names,
                                  middelberg_component_swires, middelberg_component_source_ras,
                                  middelberg_component_source_decs, middelberg_component_source_zs,
],
                            names=columns)
table

Key,Component ID (Norris),Source ID (Norris),Source Name (Norris),SWIRE Name (Norris),RA (Norris),Dec (Norris),Source RA (Norris),Source Dec (Norris),Component ID (RGZ),Zooniverse ID (RGZ),SWIRE Name (RGZ),RA (RGZ),Dec (RGZ),Source ID (Fan),SWIRE Name (Fan),Component ID (Middelberg),Component Name (Middelberg),RA (Middelberg),Dec (Middelberg),Source ID (Middelberg),Source Name (Middelberg),SWIRE Name (Middelberg),Source RA (Middelberg),Source Dec (Middelberg),Source Redshift (Middelberg)
int32,str4,str4,str26,str32,str32,str32,str32,str32,str8,str10,str26,str32,str32,str11,str26,str4,str26,str18,str19,str4,str26,str32,str18,str19,str32
0,C001,S001,ATCDFS_J032602.78-284709.0,SWIRE3_J032603.15-284708.5,51.511604166666665,-28.785833333333336,51.511604166666665,-28.78585,CI0412,ARG0003rb2,SWIRE3_J032602.36-284711.5,51.511734,-28.785575,91,SWIRE3_J032603.15-284708.5,,,,,,,,,,
1,C002,S002,ATCDFS_J032604.15-275659.3,--,51.5173,-27.949805555555557,51.5173,-27.949830555555558,,,,,,,,,,,,,,,,,
2,C003,S003,ATCDFS_J032605.68-274734.4,--,51.5236875,-27.79288888888889,51.5236875,-27.792911111111113,,,,,,,,,,,,,,,,,
3,C004,S004,ATCDFS_J032606.95-275332.2,--,51.528979166666666,-27.892277777777778,51.528979166666666,-27.892294444444445,,,,,,,,,,,,,,,,,
4,C005,S005,ATCDFS_J032611.47-273243.8,--,51.5478125,-27.5455,51.5478125,-27.54550277777778,,,,,,,,,,,,,,,,,
5,C006,S006,ATCDFS_J032613.70-281717.7,(441298),51.557087499999994,-28.28825,51.557087499999994,-28.288252777777778,,,,,,,,,,,,,,,,,
6,C007,S007,ATCDFS_J032615.48-284629.2,SWIRE3_J032615.41-284630.7,51.5645375,-28.77477777777778,51.5645375,-28.77478888888889,CI0614,ARG0003rfr,SWIRE3_J032615.41-284630.7,51.564555,-28.774847,153,SWIRE3_J032615.41-284630.7,,,,,,,,,,
7,C008,S008,ATCDFS_J032615.55-280601.0,SWIRE3_J032615.52-280559.8,51.564820833333336,-28.10027777777778,51.564820833333336,-28.100291666666667,CI0320,ARG0003r8s,SWIRE3_J032615.52-280559.8,51.564799,-28.099955,156,SWIRE3_J032615.52-280559.8,,,,,,,,,,
8,C009,S009,ATCDFS_J032616.35-280014.6,SWIRE3_J032616.31-280014.7,51.5681375,-28.004055555555556,51.5681375,-28.004058333333333,,,,,,996,SWIRE3_J032616.31-280014.7,,,,,,,,,,


In [16]:
table.write('one-table-to-rule-them-all.tbl', format='ascii')